In [ ]:
%pip install cartopy pandas matplotlib chardet xgboost

### Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeatures
import seaborn as sns
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

# Local
from jaguar_feature_scaling import get_added_features_plot,get_temporal_analysis_plot
from helper import get_dataset_with_copy, calculate_group_directions, calculate_group_distances, remove_outliers, create_time_window_features,calculate_movement_features, classify_movement_state

### Pandas configuration
Configure pandas display settings to show more data in our notebook outputs
This helps us see more rows and columns when examining our dataframes, rather than having them truncated with ellipsis (...) 

In [ ]:
# This will be set to see most of the infomation of any print that i make
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000); 

### File reading and copy variable declaration
We create two versions of our dataframe: main and copy
The copy preserves our original, untouched data as a backup reference, while the main dataframe will be used for active analysis and transformations.
This is particularly useful in Jupyter notebooks where we can always refer back to the original state of our data without reloading the file or having to restart the notebook to run it all again

In [ ]:
# Detect the file encoding
jaguar_data_original, jaguar_data = get_dataset_with_copy('datasets/jaguar_movement_data.csv')

jaguar_info_original, jaguar_info = get_dataset_with_copy('datasets/jaguar_additional_information_2.csv')

### Data Visualization

#### Initial Data Exploration
Display basic information about both datasets including their structure and dimensions

In [ ]:
print(jaguar_data_original.head())
print(jaguar_data_original.shape)
print(jaguar_info_original.head())
print(jaguar_info_original.shape)

Check detailed information about data types and null values in both datasets

In [ ]:
print(jaguar_data_original.info())
print(jaguar_info_original.info())

#### Gender Analysis
Analyze and visualize gender distribution

In [ ]:
print(jaguar_info_original['Sex'].unique())
gender = jaguar_info_original['Sex'].value_counts()

Create bar plot for gender distribution

In [ ]:
gender.plot(kind='bar',figsize=(10,10))
plt.title('Number of birds detected')
plt.xlabel('Sex')
plt.ylabel('Number')
plt.show()

#### Data Quality Check
Check for missing values in both datasets

In [ ]:
print(jaguar_data_original.isnull().sum())
print("---------")
print(jaguar_info_original.isnull().sum())

Check unique values in both datasets

In [ ]:
print(jaguar_data_original.nunique())
print("---------")
print(jaguar_info_original.nunique())

#### Missing Value Treatment
Since there are null values in the columns "Estimated Age" and "Weight" (jaguar_info) we will be filling them 0.

In [ ]:
jaguar_info["Estimated Age"]= jaguar_info["Estimated Age"].fillna(value=0)
jaguar_info["Weight"]= jaguar_info["Weight"].fillna(value=0)

### Data Preprocessing

Display all columns in both datasets

In [1]:
print(jaguar_data.columns.to_list())
print(jaguar_info.columns.to_list())

NameError: name 'jaguar_data' is not defined

Rename columns and drop unnecessary ones.
We remove unnecesary or unique value columns

In [ ]:
jaguar_data.rename(columns={'location.long': 'longitude', 'location.lat': 'latitude', 'individual.local.identifier (ID)': 'individual_id' }, inplace=True)
jaguar_data.drop(['Event_ID', 'individual.taxon.canonical.name','tag.local.identifier', 'study.name', 'country'], axis=1,inplace=True)
jaguar_info.rename(columns={'ID':'individual_id','Sex': 'sex', 'Estimated Age': 'age', 'Weight': 'weight' }, inplace=True)
jaguar_info.drop(['Collar Type', 'Collar Brand','Planned Schedule', 'Project Leader', 'Contact'], axis=1,inplace=True)

Convert timestamp to datetime.

In [ ]:
jaguar_data['timestamp'] = pd.to_datetime(jaguar_data['timestamp'], errors='coerce')
print(jaguar_data.dtypes)

Display all the columns in both datasets to confirm the changes we've done.

In [ ]:
print(jaguar_data.columns.to_list())
print(jaguar_info.columns.to_list())

#### Data Merge and Grouping

Merge datasets.
We merge them using the id of the individual.

In [ ]:
jaguar_datanew = jaguar_data.merge(jaguar_info, on='individual_id', how='left')
print(jaguar_datanew)

Grouping. We create a dictionary where each key is the id of the jaguar and the value is their tracking and information data.


In [ ]:
jaguar_groups = {individual_id: group for individual_id, group in jaguar_datanew.groupby('individual_id')}
print(jaguar_groups)

Print record count for each jaguar

In [ ]:
for jaguar_id, subset in jaguar_groups.items():
    print(f"Jaguar {jaguar_id}: {len(subset)} records")

### Feature Engineering

#### Time Features
Adding time-based features

In [ ]:
jaguar_datanew['hour'] = jaguar_datanew['timestamp'].dt.hour
jaguar_datanew['day'] = jaguar_datanew['timestamp'].dt.day
jaguar_datanew['month'] = jaguar_datanew['timestamp'].dt.month
jaguar_datanew['year'] = jaguar_datanew['timestamp'].dt.year
jaguar_datanew['dayofweek'] = jaguar_datanew['timestamp'].dt.dayofweek

Create time period categories:
- 0 (24) to 6 : Night
- 6 to 12: Morning
- 12 to 18: Afternoon
- 18 to 24 (0): Evening

In [ ]:
jaguar_datanew['time_of_day'] = pd.cut(jaguar_datanew['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])

#### Movement Analysis Features

We will be calculating:
- Time differences between consecutive points for each jaguar
- Distances
- Speed
- Direction of movement

In [ ]:
jaguar_datanew['time_diff'] = jaguar_datanew.groupby('individual_id')['timestamp'].diff()
jaguar_datanew['distance'] = jaguar_datanew.groupby('individual_id', group_keys=False).apply(calculate_group_distances)
jaguar_datanew['time_diff_hours'] = jaguar_datanew['time_diff'].dt.total_seconds() / 3600
jaguar_datanew['speed'] = jaguar_datanew['distance'] / jaguar_datanew['time_diff_hours'].replace({0: np.nan})
jaguar_datanew['direction'] = jaguar_datanew.groupby('individual_id', group_keys=False).apply(calculate_group_directions)

#### Data Cleaning and Validation
Handle infinite values and outliers

In [ ]:
jaguar_datanew = jaguar_datanew.replace([np.inf, -np.inf], np.nan)
jaguar_datanew = remove_outliers(jaguar_datanew, 'speed')
jaguar_datanew = remove_outliers(jaguar_datanew, 'distance')

Fill missing values

In [ ]:
jaguar_datanew['speed'] = jaguar_datanew['speed'].fillna(method='ffill')
jaguar_datanew['distance'] = jaguar_datanew['distance'].fillna(method='ffill')
jaguar_datanew['direction'] = jaguar_datanew['direction'].fillna(method='ffill')

#### Results Validation
Display processed data sample

In [ ]:
print(jaguar_datanew[['individual_id', 'timestamp', 'latitude', 'longitude', 'distance', 'speed', 'direction']].head(10))

Display summary statistics

In [ ]:
print(jaguar_datanew['distance'].describe())
print("-----")
print(jaguar_datanew['speed'].describe())
print("-----")
print(jaguar_datanew['direction'].describe())


#### Ploting statistics

In [ ]:

features_plt = get_added_features_plot(jaguar_datanew)
features_plt.show()

##### Additional temporal analysis

In [ ]:
# Additional temporal analysis
travaled_distance_plt = get_temporal_analysis_plot(jaguar_datanew)
plt.show()

Individual jaguar movement patterns

In [ ]:
fig, ax = plt.subplots(figsize=(40, 8), subplot_kw={'projection': ccrs.PlateCarree()})
# Set the extent to the whole world (-180 to 180 longitude, -90 to 90 latitude)
# ax.set_global()

# Add map features
ax.add_feature(cfeatures.LAND, edgecolor='black')
ax.add_feature(cfeatures.OCEAN)
ax.add_feature(cfeatures.COASTLINE)
ax.add_feature(cfeatures.BORDERS, linestyle=':')

# Plot each jaguar's movement

for jaguar_id in jaguar_datanew['individual_id'].unique():
    jaguar_subset = jaguar_datanew[jaguar_datanew['individual_id'] == jaguar_id]
    ax.plot(jaguar_subset['longitude'], jaguar_subset['latitude'], 
             label=f'Jaguar {jaguar_id}', alpha=1)

# Labels and legend
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Jaguar Movement Paths on World Map")
ax.legend()
plt.show()

Summary Statistics

In [ ]:
print("\nMovement Statistics by Time of Day:")
print(jaguar_datanew.groupby('time_of_day')[['speed', 'distance']].agg(['mean', 'std']).round(3))

print("\nMovement Statistics by Individual:")
print(jaguar_datanew.groupby('individual_id')[['speed', 'distance']].agg(['mean', 'std']).round(3))

# Correlations
As we can see in the plotted correlation matrix underneath:
There is a strong correlation between the following columns:
    - Latitude and longitude (Negative)
There is also moderate correlation between:
    - Timestamp and latitude (positive)
There is a weak or no correlation at all between (values closer to 0 both negative and positive):
    - Individual id and latitude
    - Individual id and longitude
    - Timestamp and longitude



In [ ]:
numeric_columns = jaguar_datanew.select_dtypes(include=[np.number]).columns
correlation_data = jaguar_datanew[numeric_columns].corr()
plt.figure(figsize=(20, 10))
sns.heatmap(correlation_data,  annot=True)

#Filtering Out Self-Correlations
# Unstack the correlation matrix
correlation_pairs = correlation_data.unstack()

# Filter out self-correlations (where feature pairs are the same)
filtered_correlation_pairs = correlation_pairs[correlation_pairs.index.get_level_values(0) != correlation_pairs.index.get_level_values(1)]

# Sort the remaining pairs in descending order of correlation
filtered_correlation_pairs = filtered_correlation_pairs.sort_values(kind="quicksort", ascending=True)

print(filtered_correlation_pairs)

In [ ]:

# for jaguar_id, subset in jaguar_groups.items():
#     ax1 = subset.head().plot.scatter(x='timestamp',
#                        y='longitude',
#                        c='DarkBlue')
    #print(subset.corr())

#### Splitting dataset (unsed)

In [ ]:
jaguar_data_copy = jaguar_data.copy()
X_copy = jaguar_data_copy.drop('individual_id', axis=1)
X_copy.shape


In [ ]:
y_copy = jaguar_data_copy['individual_id']         # we want to predict y using X
y_copy.shape

In [ ]:
#1 - Split the dataset: tes=25%; training=75%
# test size=25%
X_train, X_test, y_train, y_test = train_test_split(X_copy,y_copy,test_size=0.25,random_state=40)

print(len(X_train)*100/len(jaguar_data_copy))

### Feature Scaling

In [ ]:
window_features = []
for jaguar_id in jaguar_datanew['individual_id'].unique():
    # Get data for current jaguar
    jaguar_data = jaguar_datanew[jaguar_datanew['individual_id'] == jaguar_id].copy()
    
    # Calculate window features without setting index beforehand
    window_stats = create_time_window_features(jaguar_data)
    window_stats['individual_id'] = jaguar_id
    window_features.append(window_stats)
    
window_features_df = pd.concat(window_features).reset_index()

window_features_df = calculate_movement_features(window_features_df)

# 3. Add temporal context features
window_features_df['hour'] = window_features_df['timestamp'].dt.hour
window_features_df['is_night'] = (window_features_df['hour'] >= 18) | (window_features_df['hour'] <= 6)
window_features_df['is_peak_activity'] = window_features_df['hour'].isin([5,6,7,17,18,19])


In [ ]:
window_features_df['movement_state'] = window_features_df.apply(classify_movement_state, axis=1)

# Let's visualize our new features
plt.figure(figsize=(15, 30))

# Plot 1: Movement States Distribution
plt.subplot(4, 1, 1)
sns.countplot(data=window_features_df, x='movement_state')
plt.title('Distribution of Movement States')
plt.xticks(rotation=45)

# Plot 2: Speed vs Area Covered
plt.subplot(4, 1, 2)
sns.scatterplot(data=window_features_df, x='speed_mean', y='area_covered', 
                hue='movement_state', alpha=0.6)
plt.title('Speed vs Area Covered by Movement State')

# Plot 3: Movement Patterns by Time of Day
plt.subplot(4, 1, 3)
sns.boxplot(data=window_features_df, x='hour', y='movement_intensity')
plt.title('Movement Intensity by Hour')
plt.xticks(rotation=45)

# Plot 4: Path Efficiency Distribution
plt.subplot(4, 1, 4)
sns.boxplot(data=window_features_df, x='movement_state', y='path_efficiency')
plt.title('Path Efficiency by Movement State')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Print summary statistics of our engineered features
print("\nSummary Statistics of Engineered Features:")
print(window_features_df.describe().round(3))

print("\nMovement State Distribution:")
print(window_features_df['movement_state'].value_counts(normalize=True).round(3))

## ML Pipeline

In [ ]:
# First, let's prepare our features and target
# Drop non-feature columns and handle any remaining NaN values
feature_columns = ['speed_mean', 'speed_max', 'speed_std', 'distance_sum', 'distance_mean', 'direction_mean', 'direction_std', 'area_covered', 'movement_intensity', 'path_efficiency', 'direction_variability', 'hour', 'is_night', 'is_peak_activity']

X = window_features_df[feature_columns].fillna(0)
y = window_features_df['movement_state']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Create dictionary of pipelines with different models
# pipelines = {
#     'random_forest': Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
#     ]),
    
#     'svm': Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', SVC(kernel='rbf', probability=True))
#     ]),
    
#     'logistic': Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', LogisticRegression(max_iter=1000))
#     ]),
    
#     'xgboost': Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', xgb.XGBClassifier(random_state=42))
#     ])
# }

pipelines = {
    'random_forest': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(n_estimators=50, random_state=42))  # reduced from 100
    ]),
    'logistic': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(max_iter=500))
    ])
}

In [ ]:
# Train and evaluate each model
results = {}
for name, pipeline in pipelines.items():
    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # Get predictions
    y_pred = pipeline.predict(X_test)
    
    # Store results
    results[name] = {
        'pipeline': pipeline,
        'train_score': pipeline.score(X_train, y_train),
        'test_score': pipeline.score(X_test, y_test),
        'cv_scores': cross_val_score(pipeline, X_train, y_train, cv=5),
        'predictions': y_pred,
        'classification_report': classification_report(y_test, y_pred)
    }

In [ ]:

# Print results
for name, result in results.items():
    print(f"\n{name.upper()} RESULTS:")
    print(f"Training Score: {result['train_score']:.4f}")
    print(f"Test Score: {result['test_score']:.4f}")
    print(f"Cross-validation Scores: {result['cv_scores'].mean():.4f} (+/- {result['cv_scores'].std() * 2:.4f})")
    print("\nClassification Report:")
    print(result['classification_report'])

# Visualize results
plt.figure(figsize=(15, 10))

# Plot 1: Model Comparison
plt.subplot(2, 2, 1)
model_scores = {name: result['test_score'] for name, result in results.items()}
plt.bar(model_scores.keys(), model_scores.values())
plt.title('Model Test Scores Comparison')
plt.xticks(rotation=45)
plt.ylabel('Accuracy')

# Plot 2: Cross-validation Scores
plt.subplot(2, 2, 2)
cv_means = [result['cv_scores'].mean() for result in results.values()]
cv_stds = [result['cv_scores'].std() for result in results.values()]
plt.errorbar(results.keys(), cv_means, yerr=cv_stds, fmt='o')
plt.title('Cross-validation Scores with Standard Deviation')
plt.xticks(rotation=45)
plt.ylabel('CV Score')

# Plot 3: Feature Importance (for Random Forest)
rf_pipeline = results['random_forest']['pipeline']
rf_classifier = rf_pipeline.named_steps['classifier']
importances = pd.Series(
    rf_classifier.feature_importances_,
    index=feature_columns
)
plt.subplot(2, 2, 3)
importances.sort_values().plot(kind='barh')
plt.title('Feature Importance (Random Forest)')

# Plot 4: Confusion Matrix for best model
best_model_name = max(results.items(), key=lambda x: x[1]['test_score'])[0]
best_predictions = results[best_model_name]['predictions']
plt.subplot(2, 2, 4)
sns.heatmap(
    confusion_matrix(y_test, best_predictions, normalize='true'),
    annot=True,
    fmt='.2f',
    xticklabels=rf_classifier.classes_,
    yticklabels=rf_classifier.classes_
)
plt.title(f'Confusion Matrix ({best_model_name})')

plt.tight_layout()
plt.show()

# Save the best model
best_model = results[best_model_name]['pipeline']

## Model Selection & Evaluation:

In [ ]:
# 1. Save the best model (Random Forest)
import pickle

best_model = results['random_forest']['pipeline']

# Save the model
with open('jaguar_behavior_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# 2. Create a prediction function
def predict_jaguar_behavior(jaguar_data):
    """
    Predict jaguar behavior from movement data
    
    Parameters:
    jaguar_data: DataFrame with required features:
        - speed_mean, speed_max, speed_std
        - distance_sum, distance_mean
        - direction_mean, direction_std
        - area_covered, movement_intensity
        - path_efficiency, direction_variability
        - hour, is_night, is_peak_activity
    
    Returns:
    predicted_behavior: string indicating behavior class
    """
    # Ensure all required features are present
    required_features = [
        'speed_mean', 'speed_max', 'speed_std',
        'distance_sum', 'distance_mean',
        'direction_mean', 'direction_std',
        'area_covered', 'movement_intensity',
        'path_efficiency', 'direction_variability',
        'hour', 'is_night', 'is_peak_activity'
    ]
    
    missing_features = [f for f in required_features if f not in jaguar_data.columns]
    if missing_features:
        raise ValueError(f"Missing required features: {missing_features}")
    
    # Make prediction
    prediction = best_model.predict(jaguar_data[required_features])
    
    return prediction

# 3. Create example usage/demo
def create_demo_prediction():
    """
    Demonstrate model usage with example data
    """
    # Create sample data
    sample_data = pd.DataFrame({
        'speed_mean': [1.5],
        'speed_max': [3.0],
        'speed_std': [0.5],
        'distance_sum': [2.0],
        'distance_mean': [0.5],
        'direction_mean': [180.0],
        'direction_std': [45.0],
        'area_covered': [1.0],
        'movement_intensity': [3.0],
        'path_efficiency': [0.8],
        'direction_variability': [30.0],
        'hour': [12],
        'is_night': [0],
        'is_peak_activity': [1]
    })
    
    # Get prediction
    behavior = predict_jaguar_behavior(sample_data)
    
    print(f"Predicted behavior: {behavior[0]}")
    
    # Get prediction probabilities
    proba = best_model.predict_proba(sample_data)
    behaviors = best_model.classes_
    
    print("\nPrediction probabilities:")
    for behavior, prob in zip(behaviors, proba[0]):
        print(f"{behavior}: {prob:.2%}")

# 4. Add real-time prediction capability
def process_new_movement_data(lat, lon, timestamp, previous_data=None):
    """
    Process new movement data point and predict behavior
    
    Parameters:
    lat: float - latitude
    lon: float - longitude
    timestamp: datetime - time of observation
    previous_data: DataFrame - previous movement data for context
    
    Returns:
    behavior: string - predicted behavior
    """
    # Calculate required features from new data point
    hour = timestamp.hour
    is_night = 1 if (hour >= 18 or hour <= 6) else 0
    is_peak_activity = 1 if hour in [5,6,7,17,18,19] else 0
    
    # If we have previous data, calculate movement features
    if previous_data is not None:
        # Calculate speed, distance, direction etc.
        pass
    else:
        # Use default values when no history is available
        pass
    
    # Create feature DataFrame
    new_data = pd.DataFrame({
        # Add features here based on calculations
    })
    
    return predict_jaguar_behavior(new_data)

# Example usage
if __name__ == "__main__":
    print("Demonstrating jaguar behavior prediction model:")
    create_demo_prediction()
    
    # Test loading saved model
    print("\nTesting model loading:")
    with open('jaguar_behavior_model.pkl', 'rb') as f:
        loaded_model = pickle.load(f)
    print("Model loaded successfully!")

## Add behavioral prediction